<a href="https://colab.research.google.com/github/koll-ai/control-meme-api/blob/main/controlmeme_client.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title # Install dependencies

!git clone https://github.com/lllyasviel/ControlNet.git
!git clone https://github.com/koll-ai/control-meme-api.git

import sys
sys.path.insert(0, '/content/control-meme-api')

!pip install --pre -U triton
!pip install safetensors
!pip install xformers
!pip install -r /content/control-meme-api/requirements.txt
!npm install -g localtunnel

In [2]:
import torch
import cv2
import numpy as np
from PIL import Image

from flask import Flask
from flask import request
import base64
from io import BytesIO

import subprocess
import time
from flask_cors import CORS

import requests

# Download model

In [4]:
model = "control_sd15_canny.pth" #@param ["control_sd15_mlsd.pth", "control_sd15_hed.pth", "control_sd15_seg.pth", "control_sd15_depth.pth", "control_sd15_scribble.pth", "control_sd15_normal.pth", "control_sd15_openpose.pth", "control_sd15_canny.pth"]

!curl -Lo $model https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/$model
!mv $model /content/ControlNet/models/$model

#detectors = ['body_pose_model.pth', 'dpt_hybrid-midas-501f0c75.pt', 'hand_pose_model.pth', 'mlsd_large_512_fp32.pth', 'mlsd_tiny_512_fp32.pth', 'network-bsds500.pth', 'upernet_global_small.pth']
#for d in detectors:
#  !curl -Lo $d   https://huggingface.co/lllyasviel/ControlNet/resolve/main/annotator/ckpts/$d
#  !mv $d /content/ControlNet/annotator/ckpts/$d

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1141  100  1141    0     0   3989      0 --:--:-- --:--:-- --:--:--  3989
100 5446M  100 5446M    0     0   180M      0  0:00:30  0:00:30 --:--:--  231M


# Load model

In [ ]:
%cd /content/ControlNet

import controlmeme

import importlib
importlib.reload(controlmeme)

controlmeme.load_model('./models/control_sd15_canny.pth')

In [6]:
import cv2 as cv

def create_hint(image, hint_type):
  init_image = image
  
  if hint_type == 'canny':
    controlnet_hint = Image.fromarray(cv.Canny(np.array(init_image), 100,200))
  
  controlnet_hint.save('last_hint.jpeg')

  return controlnet_hint.convert('RGB')

def generate_controlnet(prompt, hint, num_inference_steps, seed=-1, negative_prompt="", **kwargs):
    """
    Generate a meme variation
    POST params: prompt:str, negative_prompt:str, controlnet_hint_url:str, num_inference_steps:int, seed:int
    """
    
    hint = np.asarray(hint)

    output = controlmeme.generate(hint, prompt, "", negative_prompt, 1, 512, num_inference_steps, False, 1, 9, seed, 0)

    return Image.fromarray(output[0])

In [ ]:
with open('output.txt', 'w') as f:
    proc = subprocess.Popen(['lt', '--port', '5000'], stdout=f)

time.sleep(3)

app = Flask(__name__)
CORS(app)

params = dict()

API_URL = ""

@app.route('/hello/')
def hello():    
    return "hello"

@app.route('/', methods=['POST'])
def index():
    global params

    params = request.get_json()

    meme_url = params['controlnet_hint_url']
    image = Image.open(requests.get(meme_url, stream=True).raw)
    hint = create_hint(image, 'canny')

    last_image = generate_controlnet(hint=hint, **params)

    with open('./last_meme.jpeg', 'w') as f:
      image.save(f, format="JPEG")

    buffered = BytesIO()
    last_image.save(buffered, format="JPEG")

    return base64.b64encode(buffered.getvalue())


@app.route('/save_last/')
def save_last():
    requests.post(f"{API_URL}/api/meme/{params['uuid']}/variation/",
                  files=dict(
                      file=open('./last.jpeg'),
                      prompt=params["prompt"],
                      nb_steps=params["num_inference_steps"]
                      )
                  )    
    return "ok"

if __name__ == '__main__':
  with open('output.txt', 'r') as f:
      contents = f.read()
      API_URL = contents.split(' ')[-1]
      protocol = API_URL.split(':')[0]
      url = API_URL.split('/')[-1]

      print(API_URL)

      print('\n'*2)
      print(f'Everything is ready! Click on the this link to be redirected to koll.ai. Don\'t close this tab!')
      print(f'https://meme.koll.ai?protocol={protocol}&url={url}')

  app.run()

In [ ]:
meme_url = "https://storage.googleapis.com/control-meme-public/gigachad.jpg"

image = Image.open(requests.get(meme_url, stream=True).raw)
image

#get controlnet hint image
hint = create_hint(image, 'canny')
hint

params = {"prompt": "Gigachad Thanos",
          "hint": hint,
          "num_inference_steps": 30,
          "seed": -1
        }

b64 = generate_controlnet(**params)
b64